In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [ ]:
df = pd.read_csv("../data/processed/datos_limpios_diagnosticos.csv",
                 sep=";", encoding="latin-1")


# 1. Análisis de contenido de detalle de atención


In [ ]:
atencion_prueba = df["detalle_atencion"][0]


## Tokenizacion de Oraciones y Palabras

In [ ]:
# Tokeniza por oracion y palabras
separado_por_oracion = sent_tokenize(atencion_prueba)
separado_por_palabra = word_tokenize(atencion_prueba)

print(separado_por_oracion)
print(separado_por_palabra)

## Filtro por StopWords

In [ ]:
stop_words = stopwords.words("spanish")
atencion_sin_stopwords = [palabra for palabra in separado_por_palabra if palabra not in stop_words]
print(atencion_sin_stopwords)

## Stemming

In [ ]:
# I was taking a ride in the car.
# I was riding  in the car.
# En este caso, ride y riding significan lo mismo. Con stemming se reduce a la raiz de una palabra.

In [ ]:
from nltk.stem import PorterStemmer

ps = PorterStemmer()

In [ ]:
palabras_prueba = ["python", "pythoner", "pythoning", "pythoned", "pythonly"]
for w in palabras_prueba:
    print(ps.stem(w))

In [ ]:
nuevo_texto = (
    "It is very important to be pythonly while you are pythoning with python. "
    "All pythoners have pythoned poorly at least once."
)

palabras = word_tokenize(nuevo_texto)
for palabra in palabras:
    print(ps.stem(palabra))

## Part of Speech

Agrega una etiqueta a cada palabra, indicando si la palabra es un articulo, sustantivo, adjetivo,
etc.

In [ ]:
import nltk

from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer

- CC: It is the conjunction of coordinating
- CD: It is a digit of cardinal
- DT: It is the determiner
- EX: Existential
- FW: It is a foreign word
- IN: Preposition and conjunction
- JJ: Adjective
- JJR and JJS: Adjective and superlative
- LS: List marker
- MD: Modal
- NN: Singular noun
- NNS, NNP, NNPS: Proper and plural noun
- PDT: Predeterminer
- WRB: Adverb of wh
- WP$: Possessive wh
- WP: Pronoun of wh
- WDT: Determiner of wp
- VBZ: Verb
- VBP, VBN, VBG, VBD, VB: Forms of verbs
- UH: Interjection
- TO: To go
- RP: Particle
- RBS, RB, RBR: Adverb
- PRP, PRP$: Pronoun personal and professional

In [ ]:
palabras_con_tag = nltk.pos_tag(atencion_sin_stopwords)
print(palabras_con_tag)

## Chunking

In [ ]:
chunkgram = r""""Chunk: {<NNP.?>+<NN.?>}"""
chunkParser = nltk.RegexpParser(chunkgram)
chunked = chunkParser.parse(palabras_con_tag)

In [ ]:
chunked

## Chinking

Indica todo lo que se quiere dejar fuera de los chunks

In [ ]:
chunkgram = r""""Chunk: {<.*>+}
                        }<VB.?|IN|DT>+{"""
chunkParser = nltk.RegexpParser(chunkgram)
chunked = chunkParser.parse(palabras_con_tag)

In [ ]:
chunked

## Named Entity Recognition

Identifica entidades dentro de un texto. Las entidades corresponden a organizaciones (Un Hospital),
personas, fechas, porcentajes, localizaciones, entre otras.

In [ ]:
entidades_en_texto = nltk.ne_chunk(palabras_con_tag, binary=True)

In [ ]:
entidades_en_texto

## Lemmatizing

Es como el Stemming, pero las palabras finales son palabras reales.

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
print(lemmatizer.lemmatize("cats"))
print(lemmatizer.lemmatize("cacti"))
print(lemmatizer.lemmatize("geese"))
print(lemmatizer.lemmatize("rocks"))
print(lemmatizer.lemmatize("python"))

In [ ]:
print(lemmatizer.lemmatize("better", pos="a"))

Aqui se puede ver que better se transformo a good, ya que es un adverbio.

## WordNet

Se pueden tomar palabras y buscar sinónimos y antónimos!

In [ ]:
from nltk.corpus import wordnet

In [ ]:
# Se pueden buscar sinonimos de una palabra

syns = wordnet.synsets("good")
print(syns)

# Solo el nombre
print(syns[0].lemmas())

# Definicion de la palabra
print(syns[0].definition())

# Ejemplos
print(syns[0].examples())

In [ ]:
# Se pueden buscar antonimos de una palabra

for sinonimo in syns:
    for lema in sinonimo.lemmas():        
        print(lema)
        if lema.antonyms():
            print(f"   |----{lema.antonyms()}")

## Bag of Words

Existen los unigramas (toma las palabras por si solo)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
X_train = ["I love the book book", "This is a great book", "The fit is great", "I love the shoes"]
y_train = ["BOOKS", "BOOKS", "CLOTHING", "CLOTHING"]

In [ ]:
vectorizador = CountVectorizer()
resultado_vectorizado = vectorizador.fit_transform(X_train)

In [ ]:
print(vectorizador.get_feature_names_out())
print(resultado_vectorizado.toarray())

In [ ]:
from sklearn.svm import SVC

In [ ]:
modelo = SVC(kernel="linear")
modelo.fit(resultado_vectorizado, y_train)

In [ ]:
X_test = ["This is such a bad book", "I like this shoes!"]
X_test_vectores = vectorizador.transform(X_test)

In [ ]:
modelo.predict(X_test_vectores)

### Bigrama

Aqui toma conjuntos de dos palabras y las hace un vector!

In [ ]:
vectorizador = CountVectorizer(ngram_range=(1, 2))
resultado_vectorizado = vectorizador.fit_transform(X_train)

print(vectorizador.get_feature_names_out())
print(resultado_vectorizado.toarray())

Uno de los problemas ocurre cuando el conjunto de prueba contiene una palabra que nunca apareció
en el conjunto de entrenamiento. En tal caso, el modelo deja de tomar en cuenta esta palabra.

## Word of Vectors

Permite agrupar palabras que semánticamente son similares, por ejemplo: "Rojo", "Azul", "Verde" todos los
agrupa en "Color". Otro ejemplo es: "Best book I've read in year", donde uno da cuenta de que "read"
siempre aparece cerca de book. Debido a la relación anterior, se genera un vector en el espacio
que agrupa estos términos juntos.

In [ ]:
import spacy

In [ ]:
nlp = spacy.load("es_core_news_md")

In [ ]:
docs = [nlp(text) for text in X_train]
X_train_wv_vector = [X.vector for X in docs]

In [ ]:
# Representa cada oracion en muchos vectores
print(docs[0].vector)

In [ ]:
svm = SVC(kernel="linear")
svm.fit(X_train_wv_vector, y_train)

In [ ]:
X_test = ["This is such an awesome book!", "This zapato is not that great"]
X_test_vectores = [nlp(text).vector for text in X_test]

In [ ]:
svm.predict(X_test_vectores)

## Regexes

In [ ]:
import re

In [ ]:
regexp = re.compile(r"^ab[^\s]*cd$")

In [ ]:
phrases = ["abcd", "xxxx", "abxxxxcd", "xxabcdxx", "abklasduoiasduoqwduocd"]
for phrase in phrases:
    if re.match(regexp, phrase):
        print(phrase)

In [ ]:
# ejemplo_borde = ["aaa xxabcdxx ccc"]
# for phrase in ejemplo_borde:
#     if re.match(regexp, phrase):
#         print(phrase)


# Aqui se utiliza search en vez de match. Lo que hace es buscar en cada palabra de un conjunto de strings
regexp = re.compile(r"ab[^\s]*cd")
ejemplo_borde = ["aaa xxabcdxx ccc"]
for phrase in ejemplo_borde:
    if re.search(regexp, phrase):
        print(phrase)

## Stemming/Lemmatization

Es una técnica para normalizar texto! Reduce todo a su forma basal. Lemmatizing garantiza que
la palabra reducida tiene sentido, mientras que stemming puede que no.

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [ ]:
stemmer = PorterStemmer()
phrase = "Reading the books"
phrase_tokenized = word_tokenize(phrase)

stemmed_words = [stemmer.stem(palabra) for palabra in phrase_tokenized]

In [ ]:
print(stemmed_words)

In [ ]:
from nltk.stem import WordNetLemmatizer

lema = WordNetLemmatizer()

lema_words = [lema.lemmatize(palabra, pos="v") for palabra in phrase_tokenized]

In [ ]:
print(lema_words)

Algo que es muy importante es que el lematizador espera que esté el Part of Speech de cada palabra.
De esa forma, permite reducir de forma más precisa cada palabra.

## Stopwords

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stopwords_ingles = stopwords.words("english")

In [ ]:
filtradas = [palabra for palabra in phrase_tokenized if palabra not in stopwords_ingles]

In [ ]:
print(filtradas)